# Overview

This a notebook that compares the PICSAR and WarpX results.

# Instructions

Execute the cells below one by one, by selecting them with your mouse and typing `Shift + Enter`

In [1]:
import sys
sys.path.append('../../../picsar/postproc_python_script/')
from Field import Field as PxrField
import yt
from IPython.display import clear_output
import numpy as np
from scipy.constants import c, e, epsilon_0, m_e, mu_0
from ipywidgets import interact, RadioButtons, IntSlider, fixed
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


# Functions to load and plot the fields

In [2]:
def extract_data( field, iteration, code='picsar' ):
    if code == 'picsar':
        pxr_field = PxrField('./RESULTS/%s%d.pxr' %(field, iteration) )
        return(pxr_field.f[32,:,:])

    if code == 'warpx':        
        ds = yt.load( './plt%05d/' %(iteration+1) )
        all_data_level_0 = ds.covering_grid(level=0, 
                    left_edge=[-20.e-6,-20.e-6,-20.e-6], dims=ds.domain_dimensions)

        return( all_data_level_0[field][:, :, 32].T )

In [3]:
def plot_fields( field1, field2, iteration, vmin=None, vmax=None ):
    plt.clf()
    
    plt.subplot(131)
    plt.title('PICSAR')
    data1 = extract_data(field1, iteration, 'picsar')
    plt.imshow(data1, vmin=vmin, vmax=vmax)
    plt.colorbar()
    
    plt.subplot(132)
    plt.title('WARPX')
    data2 = extract_data(field2, iteration, 'warpx')
    plt.imshow(data2, vmin=vmin, vmax=vmax)
    plt.colorbar()
    
    plt.subplot(133)
    plt.title('Difference')
    plt.imshow(data2 -data1, vmin=vmin, vmax=vmax)
    plt.colorbar()

# Compare fields

In [4]:
plot_fields('jx', 'jx', 0)

('Openning of ', './RESULTS/jx0.pxr')
xmin: -0.000020, xmax: 0.000020, nx: 64
ymin: -0.000020, ymax: 0.000020, ny: 64
zmin: -0.000020, zmax: 0.000020, nz: 64
('Total number of cells:', 262144)
('Size of file:', 2097212)


yt : [INFO     ] 2016-10-26 14:04:17,124 Parameters: current_time              = 1.20364575097e-15
yt : [INFO     ] 2016-10-26 14:04:17,125 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2016-10-26 14:04:17,126 Parameters: domain_left_edge          = [ -2.00000000e-05  -2.00000000e-05  -2.00000000e-05]
yt : [INFO     ] 2016-10-26 14:04:17,127 Parameters: domain_right_edge         = [  2.00000000e-05   2.00000000e-05   2.00000000e-05]


In [5]:
interact(plot_fields, 
         field1=RadioButtons( options=['jx', 'ex']), 
         field2=RadioButtons(options=['jx', 'Ex']), 
        iteration=IntSlider(min=0, max=19, step=1), 
        vmin=fixed(-2e13), vmax=fixed(2e13) )

('Openning of ', './RESULTS/jx0.pxr')
xmin: -0.000020, xmax: 0.000020, nx: 64
ymin: -0.000020, ymax: 0.000020, ny: 64
zmin: -0.000020, zmax: 0.000020, nz: 64
('Total number of cells:', 262144)
('Size of file:', 2097212)


yt : [INFO     ] 2016-10-26 14:04:18,490 Parameters: current_time              = 1.20364575097e-15
yt : [INFO     ] 2016-10-26 14:04:18,491 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2016-10-26 14:04:18,492 Parameters: domain_left_edge          = [ -2.00000000e-05  -2.00000000e-05  -2.00000000e-05]
yt : [INFO     ] 2016-10-26 14:04:18,493 Parameters: domain_right_edge         = [  2.00000000e-05   2.00000000e-05   2.00000000e-05]


<function __main__.plot_fields>

# Plot history of fields and compare with theory

In [6]:
def extract_history( field, code, Niters=39 ):
    history = []
    for i in range(Niters):
        data = extract_data( field, i, code )
        history.append( np.average( data[:,:32]))
    return(np.array(history))

In [7]:
# Theoretical values

# Timestep, from the Courant limit
dt_th = 40.e-6/64 / 3**.5 / c
# Density
n_e = 1.e25
# Initial velocity kick
v_0 = 0.01*c

# Derived values
j0 = n_e * e * v_0
wp = np.sqrt( n_e * e**2 / m_e / epsilon_0)
tj_th = (np.arange(39)+0.5)*dt_th
te_th = (np.arange(39)+1)*dt_th
j_th = -j0 * np.cos( wp*tj_th)
e_th = mu_0*j0*c**2/wp*np.sin( wp*te_th )

## jx field

In [8]:
jx_warpx = extract_history( 'jx', 'warpx' );
jx_picsar = extract_history( 'jx', 'picsar');
clear_output()

In [9]:
plt.figure()
plt.plot( jx_warpx, label='Warp X' )
plt.plot( jx_picsar, label='PICSAR' )
plt.plot( j_th, 'k--', label='Theory' )
plt.plot( )
plt.ylabel('Amplitude of jx')
plt.xlabel('Number of iterations')
plt.legend(loc=0)

## Ex field

In [10]:
ex_warpx = extract_history( 'Ex', 'warpx' );
ex_picsar = extract_history( 'ex', 'picsar');
clear_output()

In [11]:
plt.figure()
plt.plot( ex_warpx, label='Warp X' )
plt.plot( ex_picsar, label='PICSAR' )
plt.plot( e_th, 'k--', label='Theory' )
plt.plot( )
plt.ylabel('Amplitude of Ex')

plt.xlabel('Number of iterations')
plt.legend(loc=0)